## C S 329E HW 9

# Crossvalidation and hyperparameter selection

## Enya Liu

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [100]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [101]:
diabetes = load_diabetes(as_frame = True)
df_X = diabetes.data
s_y = diabetes.target

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [102]:
def get_linear_regression_model( df_X, s_y ):
    beta_hat = np.linalg.lstsq(df_X, s_y, rcond = False)[0]
    return beta_hat

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [103]:
dict_k_df_X = dict()
dict_k_s_y = dict()
def partition_data( df_X, s_y, k ):
    k_val = len(df_X) // k
    idx = np.arange(len(df_X))
    for i in range(1, k+1):
        np.random.shuffle(idx)
        dict_k_df_X[i] = df_X.loc[idx[:k_val+1]]
        dict_k_s_y[i] = s_y.loc[idx[:k_val+1]]
        idx = idx[k_val+1:]
    return (dict_k_df_X, dict_k_s_y)   

In [104]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [105]:
for i in range(1, 6):
    print('rows in fold', i,'is',len(dict_k_df_X[i]))

rows in fold 1 is 89
rows in fold 2 is 89
rows in fold 3 is 89
rows in fold 4 is 89
rows in fold 5 is 86


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [106]:
def get_mae(s_y, s_y_hat):
    return np.sum(np.abs(s_y - s_y_hat) / len(s_y))

In [107]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [108]:
mae = np.array([])
for k in dict_k_df_X.keys():
    # partition number key as test
    df_test = dict_k_df_X[k]
    y_test = dict_k_s_y[k]
    
    # other partitions as train
    df_train = df_X.copy().drop(df_test.index.values)
    y_train = s_y.copy().drop(y_test.index.values)
    
    beta = get_linear_regression_model(df_train, y_train)
    
    predictions = np.sum(beta * df_test, axis = 1)
    
    mae = np.append( mae, get_mae(y_test, predictions) )  

print('min', mae.min())
print('max', mae.max())
print('mean', mae.mean())

min 146.94483641337277
max 160.05424980607557
mean 154.76938517690328


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [109]:
iris = load_iris(as_frame = True)
df_X = iris.data
s_y = iris.target

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [110]:
dict_k_df_X = dict()
dict_k_s_y = dict()
def partition_data( df_X, s_y, k):
    k_val = len(df_X) // k
    idx = np.arange(len(df_X))
    for i in range(1, k+1):
        np.random.shuffle(idx)
        dict_k_df_X[i] = df_X.loc[idx[:k_val+1]]
        dict_k_s_y[i] = s_y.loc[idx[:k_val+1]]
        idx = idx[k_val+1:]
    return (dict_k_df_X, dict_k_s_y)

In [111]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [112]:
# test
for i in range(1, 6):
    print('rows in fold', i,'is',len(dict_k_df_X[i]))

rows in fold 1 is 31
rows in fold 2 is 31
rows in fold 3 is 31
rows in fold 4 is 31
rows in fold 5 is 26


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [113]:
def get_acc( s_1, s_2 ):
    acc = s_1 == s_2
    acc2 = acc.sum() / len(s_1)
    return acc2

In [114]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [115]:
import sklearn
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    # separate train and test case each time
    lst = []
    for i in range(1, 6):
        if i != k:
            lst.append(i)
    df_train = pd.concat([dict_k_df_X[lst[0]], dict_k_df_X[lst[1]], dict_k_df_X[lst[2]], dict_k_df_X[lst[3]]])
    y_train = pd.concat([dict_k_s_y[lst[0]], dict_k_s_y[lst[1]], dict_k_s_y[lst[2]], dict_k_s_y[lst[3]]])
    
    
    for pos_min_impurity in possible_min_impurity_decrease:
        
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        cross_validation = sklearn.model_selection.cross_validate(tree.DecisionTreeClassifier(min_impurity_decrease = pos_min_impurity), df_train, y_train, cv=4)
        avg_acc = cross_validation['test_score'].mean()
        train_lst = []
        train_lst.append(avg_acc)
        print('Testing',pos_min_impurity, 'min impurity decrease')
        print('    Average accuracy over 4 folds is', avg_acc)
        
    # Use best min impurity decrease to train model
    print()
    # using the highest avg acc here to find best one
    best_min_impurity = possible_min_impurity_decrease[train_lst.index(max(train_lst))]
    print('Best min impurity decrease is', best_min_impurity)
    print()
    
    # outer accuracy calculation 
    clf = tree.DecisionTreeClassifier(min_impurity_decrease = best_min_impurity)
    tree2 = clf.fit(dict_k_df_X[k], dict_k_s_y[k])
    
    this_acc = get_acc(tree2.predict(dict_k_df_X[k]), dict_k_s_y[k])

    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9410919540229885
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.7994252873563218
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.6890804597701149
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.3528735632183908

Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9580459770114943
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.8663793103448276
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.7054597701149425
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.33620689655172414

Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9410919540229884
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.8744252873563217
Testing 0.3 min impurity

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [116]:
print('min', outer_acc.min())
print('max', outer_acc.max())
print('mean', outer_acc.mean())

min 0.9615384615384616
max 1.0
mean 0.972952853598015
